In [1]:
# !pip install transformers
## Comes preinstalled

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
#         pass

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
##Use pipeline
from transformers import pipeline

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


### Sentiment Analysis classification

In [4]:
# nlp_sentence_sentiment = pipeline('sentiment-analysis')
# print(nlp_sentence_sentiment("It's a bad coffee!!"))
# print(nlp_sentence_sentiment("It's a good coffee!!"))

### Token Classification - Named Entity Recognition

In [5]:
# nlp_token_class = pipeline('ner')
# nlp_token_class('Hugging Face is a French company based in New-York.')

### Remaninig in pipeline...
https://github.com/huggingface/transformers/blob/master/notebooks/03-pipelines.ipynb

## Topic Extraction using Sem Eval 2010 dataset

In [6]:
!ls /kaggle/input/semeval-2010topic-extraction/maui-semeval2010-train | head -2

C-41.key
C-41.txt


In [7]:
import os
from nltk.tokenize import sent_tokenize

train_path = "/kaggle/input/semeval-2010topic-extraction/maui-semeval2010-train"

txt = sorted([f for f in os.listdir(train_path) if not f.endswith("-justTitle.txt") and not f.endswith(".key") and not f.endswith("-CrowdCountskey")])
key = sorted([f for f in os.listdir(train_path) if f.endswith(".key")])

txt[:3], key[:3]

(['C-41.txt', 'C-42.txt', 'C-44.txt'], ['C-41.key', 'C-42.key', 'C-44.key'])

In [8]:
filekey = dict()
for i, k in enumerate(txt):
    filekey[key[i]] = k
    
# filekey
# {'C-41.key': 'C-41.txt',
#  'C-42.key': 'C-42.txt',
#  'C-44.key': 'C-44.txt',....

In [9]:
def convert(key):
    sentences = ""
    file_path = os.path.join(train_path, filekey[key])
#     print(file_path)
    for line in open(file_path, 'r', encoding="utf8"):
        sentences += (" " + line.rstrip())
    tokens = sent_tokenize(sentences)
    key_file = open(os.path.join(train_path, str(key)),'r')
    keys = [line.strip() for line in key_file]
    key_sent = []
    labels = []
    for token in tokens:
        z = ['O'] * len(token.split())
        for k in keys:
            if k in token:
                
                if len(k.split())==1:
                    try:
                        z[token.lower().split().index(k.lower().split()[0])] = 'B'
                    except ValueError:
                        continue
                elif len(k.split())>1:
                    try:
                        if token.lower().split().index(k.lower().split()[0]) and token.lower().split().index(k.lower().split()[-1]):
                            z[token.lower().split().index(k.lower().split()[0])] = 'B'
                            for j in range(1, len(k.split())):
                                z[token.lower().split().index(k.lower().split()[j])] = 'I'
                    except ValueError:
                        continue
        for m, n in enumerate(z):
            if z[m] == 'I' and z[m-1] == 'O':
                z[m] = 'O'

        if set(z) != {'O'}:
            labels.append(z) 
            key_sent.append(token)
    return key_sent, labels

In [10]:
sentences_ = []
labels_ = []
for key, value in filekey.items():
    s, l = convert(key)
    sentences_.append(s)
    labels_.append(l)
sentences = [item for sublist in sentences_ for item in sublist]
labels = [item for sublist in labels_ for item in sublist]
# print(len(sentences), len(labels))

In [11]:
print(len(sentences), len(labels))
check_idx = 10
print(len(sentences[check_idx].split()), len(labels[check_idx]))
print(sentences[check_idx], labels[check_idx])

8584 8584
12 12
Overall, HyARM ensures efficient, predictable, and adaptive resource management for DRE systems. ['O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'O', 'O']


### Model

In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification #, BertAdam

In [13]:
MAX_LEN = 75

#batch size
bs = 32

In [14]:
tag2idx = {'B': 0, 'I': 1, 'O': 2}
tags_vals = ['B', 'I', 'O']

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device, n_gpu)

cuda 1


In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [17]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [18]:
print(tokenized_texts[check_idx])

['overall', ',', 'h', '##yar', '##m', 'ensures', 'efficient', ',', 'predictable', ',', 'and', 'adaptive', 'resource', 'management', 'for', 'dr', '##e', 'systems', '.']


In [19]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [20]:
print(tokenizer.convert_tokens_to_ids(tokenized_texts[check_idx])[:4])
print(input_ids.shape,tags.shape)
input_ids[check_idx]

[3452, 1010, 1044, 13380]
(8584, 75) (8584, 75)


array([ 3452,  1010,  1044, 13380,  2213, 21312,  8114,  1010, 21425,
        1010,  1998, 19293,  7692,  2968,  2005,  2852,  2063,  3001,
        1012,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0])

In [21]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [22]:
print(len(attention_masks), attention_masks[check_idx])

8584 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [23]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [24]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [25]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [26]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

In [27]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [28]:
if device.type =="cuda":
    print("Using  GPU")
    model = model.cuda()
else:
    print("Using CPU. Switch to GPU or TPU")

Using  GPU


In [29]:
list(model.named_parameters())[-2:]

[('classifier.weight',
  Parameter containing:
  tensor([[-0.0082,  0.0015, -0.0111,  ...,  0.0227, -0.0138, -0.0153],
          [ 0.0230,  0.0254, -0.0039,  ...,  0.0021, -0.0066, -0.0129],
          [ 0.0066, -0.0075,  0.0006,  ...,  0.0042, -0.0117,  0.0040]],
         device='cuda:0', requires_grad=True)),
 ('classifier.bias',
  Parameter containing:
  tensor([0., 0., 0.], device='cuda:0', requires_grad=True))]

In [30]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [31]:
print(len([x for x in model.named_parameters()]))
print(len(optimizer_grouped_parameters))
optimizer

201
2


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 3e-05
    weight_decay: 0
    weight_decay_rate: 0.01

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 3e-05
    weight_decay: 0
    weight_decay_rate: 0.0
)

In [32]:
device

device(type='cuda')

In [33]:
if device.type=="cpu":
    !pip install seqeval[cpu]
else:
     !pip install seqeval[gpu]

     |████████████████████████████████| 320.4 MB 23 kB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7423 sha256=83903c1de8c85a625e5b8218fc005779ced42584a5a819e7b5cbacbf0cd0655a
  Stored in directory: /root/.cache/pip/wheels/dc/cc/62/a3b81f92d35a80e39eb9b2a9d8b31abac54c02b21b2d466edc
Successfully built seqeval


In [34]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [35]:
%%time
epochs = 4
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            eval_outputs = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            tmp_eval_loss = eval_outputs[0]
            logits = eval_outputs[1]
#             logits = model(b_input_ids, token_type_ids=None,
#                            attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.18749749426506768
Validation loss: 0.14298865088710078
Validation Accuracy: 0.9781235711019661


Epoch:  25%|██▌       | 1/4 [01:09<03:29, 69.70s/it]

F1-Score: 0.28101644245142
Train loss: 0.12792086533524774
Validation loss: 0.11268970729024322
Validation Accuracy: 0.9766489483310472


Epoch:  50%|█████     | 2/4 [02:18<02:18, 69.48s/it]

F1-Score: 0.4045627376425855
Train loss: 0.09281401614820169


Epoch:  75%|███████▌  | 3/4 [03:27<01:09, 69.37s/it]

Validation loss: 0.10265495462550057
Validation Accuracy: 0.9701977594878831
F1-Score: 0.41341795104261114
Train loss: 0.07213048656925189
Validation loss: 0.09901078293720882
Validation Accuracy: 0.9695547553726565


Epoch: 100%|██████████| 4/4 [04:37<00:00, 69.27s/it]

F1-Score: 0.42679037402145553
CPU times: user 3min 9s, sys: 1min 24s, total: 4min 34s
Wall time: 4min 37s


In [36]:
print(len(pred_tags), len(pred_tags[0]))
pred_tags[0][:5]

64425 1


'O'

In [37]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        test_eval_outputs = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        tmp_eval_loss = test_eval_outputs[0]
        logits = test_eval_outputs[1]
#         logits = model(b_input_ids, token_type_ids=None,
#                        attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])

    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Validation loss: 0.09901078293720882
Validation Accuracy: 0.9695547553726565
Validation F1-Score: 0.42679037402145553


### Get keywords from sentence

In [38]:
def keywordextract(sentence):
    text = sentence
    tkns = tokenizer.tokenize(text)
#     print(len(tkns), tkns)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tkns)
    segments_ids = [0] * len(tkns)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    model.eval()
    prediction = []
    output = model(tokens_tensor, token_type_ids=None,
                                  attention_mask=segments_tensors)
    print(output)
    logit = output[0]
    logit = logit.detach().cpu().numpy()
    prediction.extend([list(p) for p in np.argmax(logit, axis=2)])
    print(prediction)
    for k, j in enumerate(prediction[0]):
        if j==1 or j==0:
            print(tokenizer.convert_ids_to_tokens(tokens_tensor[0].to('cpu').numpy())[k], j)

In [39]:
text = "The solution is based upon an abstract representation of the mobile object system."

In [40]:
keywordextract(text)

(tensor([[[-1.6967, -3.2741,  4.5658],
         [-0.3182, -3.0870,  3.0900],
         [-2.4640, -3.0383,  4.9970],
         [-2.3214, -3.0440,  4.9129],
         [-2.5085, -2.9729,  5.0386],
         [-2.3800, -3.0383,  4.9334],
         [-0.7271, -3.0077,  3.1591],
         [-0.3596, -1.9379,  1.8227],
         [-2.5585, -2.8082,  4.9467],
         [-1.6948, -3.2882,  4.6152],
         [ 2.7192, -3.1014,  0.5581],
         [-2.6877,  1.5002,  0.7933],
         [-3.3463, -2.1863,  5.0999],
         [-2.8180, -3.2575,  5.7694]]], device='cuda:0',
       grad_fn=<AddBackward0>),)
[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2]]
mobile 0
object 1


In [41]:
model.save_pretrained("bert_topic_model")

In [42]:
!ls -l 

total 3864
---------- 1 root root 3947454 Sep 28 00:54 __notebook__.ipynb
drwxr-xr-x 2 root root    4096 Sep 28 00:54 bert_topic_model
drwxr-xr-x 2 root root    4096 Sep 28 00:47 wandb


In [43]:
import transformers
model2 = BertForTokenClassification.from_pretrained("bert_topic_model")

In [44]:
def keywordextract_saved(sentence, use_model):
    text = sentence
    tkns = tokenizer.tokenize(text)
#     print(len(tkns), tkns)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tkns)
    segments_ids = [0] * len(tkns)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    use_model.eval()
    prediction = []
    output = use_model(tokens_tensor, token_type_ids=None,
                                  attention_mask=segments_tensors)
    print(output)
    logit = output[0]
    logit = logit.detach().cpu().numpy()
    prediction.extend([list(p) for p in np.argmax(logit, axis=2)])
    print(prediction)
    for k, j in enumerate(prediction[0]):
        if j==1 or j==0:
            print(tokenizer.convert_ids_to_tokens(tokens_tensor[0].to('cpu').numpy())[k], j)

In [45]:
keywordextract_saved(text, model2)

RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_index_select